### Import the libraries

In [2]:
import boto3
import pandas as pd
import awswrangler as wr
from io import StringIO
from sagemaker import image_uris, TrainingInput
from sagemaker.estimator import Estimator
import datetime
import boto3
import awswrangler as wr
import pandas as pd
import sagemaker
from sagemaker.session import Session
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.feature_store.feature_group import FeatureGroup
from io import StringIO
import datetime

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
%store

Stored variables and their in-db values:
baseline_dataset                       -> 's3://housing-dataset-5435v3/processed/housing/tra
bucket_name                            -> 'housing-dataset-5435v3'
image_uri                              -> '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgbo
model_data                             -> 's3://housing-dataset-5435v3/HousingTrain/pipeline
pipeline_definition                    -> {'Version': '2020-12-01', 'Metadata': {}, 'Paramet
role                                   -> 'arn:aws:iam::952054755114:role/LabRole'
set_up_dependencies_passed             -> True
set_up_s3_bucket_passed                -> True


In [4]:
# get bucket_name
%store -r bucket_name
print(bucket_name)

housing-dataset-5435v3


In [5]:
# save Amazon information
account_id = boto3.client("sts").get_caller_identity().get("Account")
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
s3 = boto3.client('s3', region_name=sagemaker_session.boto_region_name)

In [6]:
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

### Construct Athena queries to read the data from each offline feature store

In [7]:
train_feature_group_name = "housing_train"
validation_feature_group_name = "housing_validation"
test_feature_group_name = "housing_test"

In [8]:
# set database name and table name
database_name = "housing"
table_name = "data"

In [9]:
sagemaker_client.list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'housing_validation',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:952054755114:feature-group/housing_validation',
   'CreationTime': datetime.datetime(2024, 6, 20, 22, 29, 54, 868000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'housing_train',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:952054755114:feature-group/housing_train',
   'CreationTime': datetime.datetime(2024, 6, 20, 22, 29, 33, 574000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'housing_test',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:952054755114:feature-group/housing_test',
   'CreationTime': datetime.datetime(2024, 6, 20, 22, 30, 13, 468000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}}],
 'ResponseMetadata': {'RequestId': '9f17df37-

In [10]:
# Initialize FeatureGroup
train_feature_group = FeatureGroup(name=train_feature_group_name, sagemaker_session=sagemaker_session)

In [11]:
# Initialize FeatureGroup
validation_feature_group = FeatureGroup(name=validation_feature_group_name, sagemaker_session=sagemaker_session)

In [12]:
# Initialize FeatureGroup
test_feature_group = FeatureGroup(name=test_feature_group_name, sagemaker_session=sagemaker_session)

In [13]:
train_data_query = train_feature_group.athena_query()
validation_data_query = validation_feature_group.athena_query()
test_data_query = test_feature_group.athena_query()

In [14]:
train_data_table = train_data_query.table_name
validation_data_table = validation_data_query.table_name
test_data_table = test_data_query.table_name

In [15]:
train_query = f"""
SELECT * FROM "{train_data_table}"
"""

In [16]:
validation_query = f"""
SELECT * FROM "{validation_data_table}"
"""

In [17]:
test_query = f"""
SELECT * FROM "{test_data_table}"
"""

### Execute queries and put into dataframes

In [18]:
# run Athena query. The output is loaded to a Pandas dataframe.
df_train = pd.DataFrame()
train_data_query.run(query_string=train_query, output_location='s3://'+bucket_name+'/query_results/train/')
train_data_query.wait()
df_train = train_data_query.as_dataframe()

In [19]:
# run Athena query. The output is loaded to a Pandas dataframe.
df_validation = pd.DataFrame()
validation_data_query.run(query_string=validation_query, output_location='s3://'+bucket_name+'/query_results/validation/')
validation_data_query.wait()
df_validation = validation_data_query.as_dataframe()

In [20]:
# run Athena query. The output is loaded to a Pandas dataframe.
df_test = pd.DataFrame()
test_data_query.run(query_string=test_query, output_location='s3://'+bucket_name+'/query_results/test/')
test_data_query.wait()
df_test = test_data_query.as_dataframe()

In [21]:
df_test.head(3)

,id,mssubclass,lotarea,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,...,poolarea,miscval,mosold,yrsold,saleprice,logsaleprice,event_time,write_time,api_invocation_time,is_deleted
0,1987.0,20.0,3710.0,2007.0,2008.0,20.0,0.0,0.0,1146.0,1146.0,...,0.0,0.0,3.0,2008.0,163000.0,12.001506,1.718923e+09,2024-06-20 22:37:13.976,2024-06-20 22:32:17.000,False
1,1896.0,50.0,8212.0,1941.0,1950.0,0.0,626.0,0.0,94.0,720.0,...,0.0,0.0,9.0,2009.0,163000.0,12.001506,1.718923e+09,2024-06-20 22:37:13.976,2024-06-20 22:32:17.000,False
2,338.0,20.0,9135.0,2002.0,2003.0,113.0,810.0,0.0,726.0,1536.0,...,0.0,0.0,12.0,2008.0,214000.0,12.273731,1.718923e+09,2024-06-20 22:37:13.976,2024-06-20 22:32:17.000,False


In [22]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   292 non-null    float64
 1   mssubclass           292 non-null    float64
 2   lotarea              292 non-null    float64
 3   yearbuilt            292 non-null    float64
 4   yearremodadd         292 non-null    float64
 5   masvnrarea           292 non-null    float64
 6   bsmtfinsf1           292 non-null    float64
 7   bsmtfinsf2           292 non-null    float64
 8   bsmtunfsf            292 non-null    float64
 9   totalbsmtsf          292 non-null    float64
 10  firstflrsf           292 non-null    float64
 11  secondflrsf          292 non-null    float64
 12  lowqualfinsf         292 non-null    float64
 13  grlivarea            292 non-null    float64
 14  bsmtfullbath         292 non-null    float64
 15  bsmthalfbath         292 non-null    flo

### For df_train and df_validation the "id" column is removed and for df_test both the "id" column and "saleprice" column are removed

In [23]:
# Check if 'saleprice' column is in df_train
is_saleprice_in_train = 'saleprice' in df_train.columns
print(f"'saleprice' in df_train: {is_saleprice_in_train}")

# Check if 'saleprice' column is in df_validation
is_saleprice_in_validation = 'saleprice' in df_validation.columns
print(f"'saleprice' in df_validation: {is_saleprice_in_validation}")

# Check if 'saleprice' column is in df_validation
is_saleprice_in_test = 'saleprice' in df_test.columns
print(f"'saleprice' in df_test: {is_saleprice_in_test}")

'saleprice' in df_train: True
'saleprice' in df_validation: True
'saleprice' in df_test: True


In [24]:
df_train = df_train.drop(['id', 'event_time', 'write_time','api_invocation_time','is_deleted'], axis=1)
df_validation = df_validation.drop(['id', 'event_time', 'write_time','api_invocation_time','is_deleted'], axis=1)
# 'saleprice' has been included in the df_test for the CICD step and calculating the MSE
df_test = df_test.drop(['id', 'event_time', 'write_time','api_invocation_time','is_deleted'], axis=1)

### Check the shape of the data

In [25]:
print(df_train.shape)
print(df_validation.shape)
print(df_test.shape)

(2043, 35)
(584, 35)
(292, 35)


In [26]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2043 entries, 0 to 2042
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mssubclass     2043 non-null   float64
 1   lotarea        2043 non-null   float64
 2   yearbuilt      2043 non-null   float64
 3   yearremodadd   2043 non-null   float64
 4   masvnrarea     2043 non-null   float64
 5   bsmtfinsf1     2043 non-null   float64
 6   bsmtfinsf2     2043 non-null   float64
 7   bsmtunfsf      2043 non-null   float64
 8   totalbsmtsf    2043 non-null   float64
 9   firstflrsf     2043 non-null   float64
 10  secondflrsf    2043 non-null   float64
 11  lowqualfinsf   2043 non-null   float64
 12  grlivarea      2043 non-null   float64
 13  bsmtfullbath   2043 non-null   float64
 14  bsmthalfbath   2043 non-null   float64
 15  fullbath       2043 non-null   float64
 16  halfbath       2043 non-null   float64
 17  bedroomabvgr   2043 non-null   float64
 18  kitchena

### Check the first few rows of the data

In [27]:
print(df_train.head(5))

   mssubclass  lotarea  yearbuilt  yearremodadd  masvnrarea  bsmtfinsf1  \
0        20.0   6718.0     2001.0        2001.0        86.0       250.0   
1        50.0   6240.0     1939.0        1952.0         0.0        48.0   
2        30.0   4118.0     1941.0        1950.0         0.0         0.0   
3        20.0   6500.0     1976.0        1976.0        84.0      1033.0   
4        20.0  11024.0     2005.0        2006.0       118.0         0.0   

   bsmtfinsf2  bsmtunfsf  totalbsmtsf  firstflrsf  ...  openporchsf  \
0         0.0     1017.0       1267.0      1312.0  ...         28.0   
1         0.0      624.0        672.0       899.0  ...          0.0   
2         0.0      693.0        693.0       693.0  ...         20.0   
3         0.0       94.0       1127.0      1127.0  ...          0.0   
4         0.0     1400.0       1400.0      1400.0  ...         55.0   

   enclosedporch  threessnporch  screenporch  poolarea  miscval  mosold  \
0            0.0            0.0          0.0   

In [28]:
df_validation.head(5)

,mssubclass,lotarea,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,...,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,logsaleprice
0,50.0,7425.0,1945.0,1950.0,0.0,0.0,0.0,672.0,672.0,1195.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2008.0,163000.0,12.001506
1,160.0,1504.0,1972.0,1972.0,0.0,252.0,0.0,294.0,546.0,546.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,163000.0,12.001506
2,60.0,16259.0,2006.0,2006.0,370.0,0.0,0.0,1249.0,1249.0,1249.0,...,154.0,0.0,0.0,0.0,0.0,0.0,9.0,2006.0,342643.0,12.744445
3,20.0,11500.0,1957.0,1957.0,0.0,0.0,0.0,0.0,0.0,845.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2009.0,84000.0,11.338573
4,20.0,9600.0,1950.0,2006.0,0.0,32.0,308.0,232.0,572.0,1337.0,...,192.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,163000.0,12.001506


In [29]:
df_test.head(5)

,mssubclass,lotarea,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,...,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,logsaleprice
0,20.0,3710.0,2007.0,2008.0,20.0,0.0,0.0,1146.0,1146.0,1246.0,...,24.0,0.0,0.0,0.0,0.0,0.0,3.0,2008.0,163000.0,12.001506
1,50.0,8212.0,1941.0,1950.0,0.0,626.0,0.0,94.0,720.0,854.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2009.0,163000.0,12.001506
2,20.0,9135.0,2002.0,2003.0,113.0,810.0,0.0,726.0,1536.0,1536.0,...,74.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,214000.0,12.273731
3,60.0,9543.0,2001.0,2001.0,0.0,0.0,0.0,845.0,845.0,845.0,...,103.0,0.0,0.0,0.0,0.0,0.0,2.0,2007.0,163000.0,12.001506
4,20.0,7584.0,1953.0,1953.0,88.0,531.0,0.0,505.0,1036.0,1036.0,...,24.0,0.0,0.0,0.0,0.0,0.0,6.0,2008.0,163000.0,12.001506


### Save the data to the S3 bucket as CSV files

In [30]:
'''
!pip install botocore==1.34.129 s3transfer==0.3.0 boto3==1.34.129
!pip install s3fs
'''

'\n!pip install botocore==1.34.129 s3transfer==0.3.0 boto3==1.34.129\n!pip install s3fs\n'

In [31]:
df_train.to_csv(f"s3://{bucket_name}/processed/housing/train.csv", header=True,  index=False)

In [32]:
df_validation.to_csv(f"s3://{bucket_name}/processed/housing/validation.csv", header=True, index=False)

In [33]:
df_test.to_csv(f"s3://{bucket_name}/processed/housing/test.csv", header=True, index=False)

### Train a model using SageMaker built-in XgBoost algorithm on the training data and validate it on the validation data
1. set the container image for the XGBoost algorithm
2. set the output path
3. set the hyperparameters
4. create an estimator
5. fit the model to the training data and validate on the validation data

In [34]:
# set the container image for the XGBoost algorithm
container = image_uris.retrieve(region=region, framework="xgboost", version="latest")

In [35]:
# set the output path
output_path = f"s3://{bucket_name}/housing/model"

In [36]:
# set the hyperparameters
hyperparameters = {
    "objective": "reg:linear",
    "num_round": "100",
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "silent": "0",
}

In [37]:
# create an estimator
estimator = Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path=output_path,
    hyperparameters=hyperparameters,
)

In [38]:
# fit the model to the training data and validate on the validation data
estimator.fit(
    {
        "train": TrainingInput(
            s3_data=f"s3://{bucket_name}/processed/housing/train.csv", content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=f"s3://{bucket_name}/processed/housing/validation.csv", content_type="text/csv"
        ),
    }
)

INFO:sagemaker:Creating training-job with name: xgboost-2024-06-24-02-50-40-797


2024-06-24 02:50:41 Starting - Starting the training job...
2024-06-24 02:50:57 Starting - Preparing the instances for training...
2024-06-24 02:51:18 Downloading - Downloading input data...
2024-06-24 02:51:43 Downloading - Downloading the training image...
2024-06-24 02:52:34 Training - Training image download completed. Training in progress.
2024-06-24 02:52:34 Uploading - Uploading generated training modelArguments: train
[2024-06-24:02:52:26:INFO] Running standalone xgboost training.
[2024-06-24:02:52:26:INFO] File size need to be processed in the node: 0.49mb. Available memory size in the node: 7960.84mb
[2024-06-24:02:52:26:ERROR] Customer Error: Non-numeric value 'm' found in the header line 'mssubclass,lotarea,yearbuilt,yearremodadd,masvnrar...' of file 'train.csv'. CSV format require no header line in it. If header line is already removed, XGBoost does not accept non-numeric value in the data.
Traceback (most recent call last):
  File "/opt/amazon/lib/python3.7/site-packages/

UnexpectedStatusException: Error for Training job xgboost-2024-06-24-02-50-40-797: Failed. Reason: ClientError: Non-numeric value 'm' found in the header line 'mssubclass,lotarea,yearbuilt,yearremodadd,masvnrar...' of file 'train.csv'. CSV format require no header line in it. If header line is already removed, XGBoost does not accept non-numeric value in the data., exit code: 1

### Upload the Sagemaker Model created during our training job to the Sagemaker Model Registry

In [ ]:
# Get the current date and time
current_datetime = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
# Update the model_name variable
model_name = f"sagemaker-xgboost-{current_datetime}"
print(model_name)

In [ ]:
# Saving training job information to be used in the ML lineage module
training_job_info = estimator.latest_training_job.describe()
if training_job_info != None:
    
    # Get the model data
    model_data = training_job_info["ModelArtifacts"]["S3ModelArtifacts"]
    # Create the primary container
    primary_container = {"Image": container, "ModelDataUrl": model_data}

    # Save our model to the Sagemaker Model Registry
    create_model_response = sagemaker_client.create_model(
        ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
    )
    print(create_model_response["ModelArn"])

In [ ]:
# Inspect Training Job Details
training_job_info

### Host the trained XGBoost model as a SageMaker Endpoint

In [ ]:
endpoint_name = "housing-endpoint"
endpoint_config_name = "housing-endpoint"
# Delete the endpoint
try:
    sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
    print(f"Endpoint '{endpoint_name}' has been deleted.")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException':
        print(f"Endpoint '{endpoint_name}' does not exist.")
    else:
        print(f"Error deleting endpoint: {e}")

# Delete the endpoint configuration
try:
    sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
    print(f"Endpoint configuration '{endpoint_config_name}' has been deleted.")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException':
        print(f"Endpoint configuration '{endpoint_config_name}' does not exist.")
    else:
        print(f"Error deleting endpoint configuration: {e}")

In [ ]:
# deploy the model to an endpoint
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name="housing-endpoint",
    wait=True,
)

In [ ]:
print(predictor.endpoint_name)

In [ ]:
# Extracting information from training_job_info
model_data = training_job_info['ModelArtifacts']['S3ModelArtifacts']
image_uri = training_job_info['AlgorithmSpecification']['TrainingImage']
baseline_dataset = training_job_info['InputDataConfig'][0]['DataSource']['S3DataSource']['S3Uri']

# Store the variables
%store model_data
%store image_uri
%store baseline_dataset

### Real time inference using the deployed endpoint

In [ ]:
# make predictions using the endpoint
csv_data = df_test.to_csv(header=False, index=False)
predictions = predictor.predict(csv_data, initial_args={"ContentType": "text/csv"}).decode("utf-8")
predictions = pd.read_csv(StringIO(predictions), header=None)
predictions.head(5)

### Delete the endpoint

In [ ]:
%store role

In [ ]:
# delete the endpoint
#predictor.delete_endpoint()
#predictor.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

### Shut down notebook resources

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>